In [1]:
#!/usr/bin/env python3

import paho.mqtt.client as mqtt
import time
import pickle
import pandas as pd
# This is the Subscriber

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("topic/predict")

In [2]:
def load_model():
    # Get headers for payload
    headers = ['times_pregnant', 'glucose', 'blood_pressure', 
           'skin_fold_thick', 'serum_insuling', 'mass_index', 'diabetes_pedigree', 'age']
    # Use pickle to load in the pre-trained model
    xgb_file = open("xgb-train-model.pkl",'rb')
    xgb_model = pickle.load(xgb_file)

    # Use pickle to load in the pre-trained model
    ada_file = open("ada-train-model.pkl",'rb')
    ada_model = pickle.load(ada_file)
    return [xgb_model,ada_model]

In [3]:
def predict(in_values):
    headers = ['times_pregnant', 'glucose', 'blood_pressure', 
           'skin_fold_thick', 'serum_insuling', 'mass_index', 'diabetes_pedigree', 'age']
    
    xgb_model,ada_model = load_model()
    
    valArr = in_values.split(':')
    splitVal = valArr[0]
    
    values_arr = splitVal[1:len(splitVal)-1]
    values = [float(i) for i in values_arr.split(',')]
    input_variables = pd.DataFrame([values],
                                columns=headers, 
                                dtype=float,
                                index=['input'])
    # Get the model's prediction
    #tic = time.time()
    tic = float(valArr[1])
    xgb_prediction_proba = xgb_model.predict_proba(input_variables)
    xgb_prediction = (xgb_prediction_proba[0])[1]
    xgb_ret = float(xgb_prediction)
    
    ada_prediction_proba = ada_model.predict_proba(input_variables)
    ada_prediction = (ada_prediction_proba[0])[1]
    ada_ret = float(ada_prediction)
    toc = time.time()
    timetaken = toc - tic
    if ada_ret >= xgb_ret:
        print('{"ada-prediction":' + str(float(ada_prediction)) + '}' + ", time:" + str(timetaken))

    else:
        print('{"xgb-prediction":' + str(float(xgb_prediction)) + '}' + ", time:" + str(timetaken))


In [4]:
def on_message(client, userdata, msg):
    values = msg.payload.decode()
    #print(values)
    
    predict(values)

In [5]:
client = mqtt.Client()
client.connect("129.59.234.231",1883,60)

client.on_connect = on_connect
client.on_message = on_message

client.loop_forever()

Connected with result code 0
{"xgb-prediction":0.7945857048034668}, time:0.4227783679962158
{"ada-prediction":0.4787689894358029}, time:0.03958606719970703
{"xgb-prediction":0.5331149697303772}, time:0.0377650260925293
{"ada-prediction":0.4714174241392869}, time:0.0375669002532959
{"xgb-prediction":0.8451803922653198}, time:0.03683948516845703
{"ada-prediction":0.4801429834455223}, time:0.039968252182006836
{"ada-prediction":0.485313239032451}, time:0.03449082374572754
{"ada-prediction":0.5046722011165047}, time:0.0344386100769043
{"xgb-prediction":0.7372106313705444}, time:0.02793097496032715
{"ada-prediction":0.3266766534291773}, time:0.03822803497314453
{"ada-prediction":0.48781158273198943}, time:0.04252362251281738
{"xgb-prediction":0.8558318614959717}, time:0.03676915168762207
{"ada-prediction":0.4196589520871583}, time:0.035860538482666016
{"xgb-prediction":0.6972947120666504}, time:0.05009579658508301
{"xgb-prediction":0.5904819369316101}, time:0.04756569862365723
{"ada-predict

KeyboardInterrupt: 

In [ ]:
client.disconnect()

In [ ]:
import time
values = str("[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0]:1579992122.5956023")
predict(values)